In [6]:
import os
import rasterio
from rasterio.enums import Resampling
from rasterio.shutil import copy

def resample_directory(directory, resolution=0.05, compression='LZW'):
    # Get all GeoTIFF files in the directory
    files = [f for f in os.listdir(directory) if f.endswith('.tif')]

    for file in files:
        with rasterio.open(os.path.join(directory, file)) as src:
            # Calculate the new shape based on the new resolution
            new_shape = (int(src.shape[0] * src.res[0] / resolution),
                         int(src.shape[1] * src.res[1] / resolution))

            # Resample the raster
            data = src.read(
                out_shape=new_shape,
                resampling=Resampling.bilinear
            )

            # Update the metadata
            transform = src.transform * src.transform.scale(
                (src.width / data.shape[-1]),
                (src.height / data.shape[-2])
            )

            profile = src.profile
            profile.update(
                dtype=rasterio.float32,
                height=new_shape[0],
                width=new_shape[1],
                transform=transform,
                compress=compression
            )

            # Save the resampled raster
            with rasterio.open(os.path.join(directory, f'resampled_{file}'), 'w', **profile) as dst:
                dst.write(data)

In [7]:
#resample_directory("G:/My Drive/teaching/2024_msc_remotesensing_geoinformatics/data/photogrammetric_drone_products/confobi/")
resample_directory("G:/My Drive/teaching/2024_msc_remotesensing_geoinformatics/data/photogrammetric_drone_products/scotland/")


ZeroDivisionError: division by zero

In [10]:
import os
import rasterio
from rasterio.enums import Resampling

def resample_directory(directory, target_resolution=0.10, compression='LZW'):
    # Get all GeoTIFF files in the directory
    files = [f for f in os.listdir(directory) if f.endswith('.tif')]

    for file in files:
        with rasterio.open(os.path.join(directory, file)) as src:
            # Check if the resolution is finer than the target resolution
            if src.res[0] < target_resolution or src.res[1] < target_resolution:
                # Calculate the new shape based on the target resolution
                new_shape = (int(src.shape[0] * src.res[0] / target_resolution),
                             int(src.shape[1] * src.res[1] / target_resolution))

                # Resample the raster
                data = src.read(
                    out_shape=(src.count,) + new_shape,
                    resampling=Resampling.bilinear
                )

                # Update the metadata
                transform = src.transform * src.transform.scale(
                    (src.width / data.shape[-1]),
                    (src.height / data.shape[-2])
                )

                profile = src.profile
                profile.update(
                    dtype=rasterio.float32,
                    height=new_shape[0],
                    width=new_shape[1],
                    transform=transform,
                    compress=compression
                )

                # Save the resampled raster
                with rasterio.open(os.path.join(directory, f'resample_{file}'), 'w', **profile) as dst:
                    dst.write(data)

In [11]:
resample_directory("G:/My Drive/teaching/2024_msc_remotesensing_geoinformatics/data/photogrammetric_drone_products/scotland/")

ZeroDivisionError: division by zero

In [12]:
import os
from osgeo import gdal

def process_rasters(folder_path, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".tif"):  # Adjust the file extension if needed
            input_path = os.path.join(folder_path, filename)
            output_path = os.path.join(output_folder, filename)

            # Open the raster dataset
            dataset = gdal.Open(input_path)

            if dataset is None:
                print(f"Could not open {filename}. Skipping...")
                continue

            # Get the number of bands in the raster
            num_bands = dataset.RasterCount

            if num_bands < 3:
                print(f"{filename} has less than 3 bands. Skipping...")
                continue

            # Create a new dataset with only the first 3 bands
            driver = gdal.GetDriverByName("GTiff")
            new_dataset = driver.Create(output_path, dataset.RasterXSize, dataset.RasterYSize, 3, gdal.GDT_Float32, options=["COMPRESS=LZW"])

            # Loop through the first 3 bands and copy them to the new dataset
            for i in range(3):
                band = dataset.GetRasterBand(i + 1)  # Bands are 1-indexed
                new_band = new_dataset.GetRasterBand(i + 1)
                new_band.WriteArray(band.ReadAsArray())

            # Close datasets
            del dataset
            del new_dataset

            print(f"{filename} processed and saved.")



In [ ]:
# Example usage
folder_path = "input_folder"
output_folder = "output_folder"
process_rasters(folder_path, output_folder)